In [2]:
#Axiom to PUML
from owlready2 import *
 #PascalCase
def to_pascal_case(text):
    text = text.replace('_', ' ').replace('-', ' ')
    words = text.split()
    return ''.join([word.capitalize() for word in words])
#camelCase
def to_camel_case(prop_name):
    words = prop_name.replace("_", " ").split()
    return words[0].lower() + "".join(word.capitalize() for word in words[1:]) if words else ""

def axiom_to_puml(class_entity):
    bfo = get_namespace("http://purl.obolibrary.org/obo/")
    iofcore = get_namespace("https://spec.industrialontologies.org/ontology/core/Core/")
    """Converts an OWL equivalent class axiom to PUML notation."""
    puml_output = []
    puml_output.append("!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml")
    class_map = {}
    counter = 1

    def get_class_name(entity):
        """Extracts and maps class names with proper prefixes."""
        nonlocal counter
        if isinstance(entity, ThingClass):
            print(counter, 'processing', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            class_name = entity.name
            if class_name not in class_map:
                class_map[class_name] = f"c{counter}"
                puml_output.append(f'class({class_map[class_name]}, {prefix}{to_camel_case(bfo[entity.name].label[0]) if class_name.startswith("BFO") else class_name})')
                counter += 1
            print(puml_output)
            return class_map[class_name]

        elif isinstance(entity, Restriction):
            if isinstance(entity.property, Inverse):    
                print(counter)
                print('Restriction', entity)
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(bfo[entity.property.property.name].label[0]) if entity.property.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
            else:
                print(counter)
                print('Restriction', entity)
                prefix = "bfo:" if "BFO" in str(entity) else "iof:"
                prop_name = prefix + to_camel_case(bfo[entity.property.name].label[0]) if entity.property.name.startswith("BFO") else to_camel_case(iofcore[entity.property.name].label[0])
                print('Res Propname',prop_name)
                return process_restriction(entity, prop_name)
        
        elif isinstance(entity, And): 
            print('And', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Or): 
            print('Or', entity)
            prefix = "bfo:" if "BFO" in str(entity) else "iof:"
            prop_name = None
            return process_restriction(entity, prop_name)
        
        elif isinstance(entity, Not):
            return 
        
        elif isinstance(entity, Inverse):
            print('Inverse', entity.property)
            return

        else:
            print('error',entity)
            raise ValueError(f"Unsupported entity type: {type(entity)}")

    def process_restriction(restriction, prop_name):
        """Handles OWL 'some' restrictions including unions inside."""
        nonlocal counter
        if isinstance(restriction, And):
            print('and', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"intersection({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        
        elif isinstance(restriction, Or):
            print('and', restriction)
            entities = [get_class_name(e) for e in restriction.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities)
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name
        

        elif isinstance(restriction.value, Or):
            print('or', restriction.value)
            entities = [get_class_name(e) for e in restriction.value.Classes]
            union_name = f"ce{counter}"
            entity_list = ", ".join(f'\"{e}\"' for e in entities) 
            puml_output.append(f"union({union_name}, '[{entity_list}]')")
            counter += 1
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {union_name})")
            counter += 1
            return var_name

        else:  
            print('restriction else', restriction)
            class_name = get_class_name(restriction.value)
            var_name = f"ce{counter}"
            puml_output.append(f"some({var_name}, {prop_name}, {class_name})")
            counter += 1
            print(puml_output)
            return var_name
    
    if len(class_entity.equivalent_to) == 0:
        print('class_entity.equivalent_to',class_entity.equivalent_to)
        return
        
    else:
        print(class_entity.equivalent_to)
        axiom = class_entity.equivalent_to[0]
        main_class_name = get_class_name(class_entity)
        processed_parts = [get_class_name(part) for part in axiom.Classes]
        #change from classes to axiom and let the func figure it out
        print(processed_parts)

        if len(processed_parts) > 1:
            final_union = f"ce{counter}"
            entity_list = ", ".join(f'"{e}"' for e in processed_parts)
            puml_output.append(f"intersection({final_union}, '[{entity_list}]')")
            puml_output.append(f"equivalent({main_class_name}, {final_union})")

        return "\n".join(puml_output)

# Example Usage: [ProcuringBusinessProcess,MaterialArtifact, PieceOfEquipment], [MeasurementInformationContentEntity]
input = "MeasurementInformationContentEntity"
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
# axiom_result = axiom_to_puml(iofonto["ProcuringBusinessProcess"])
axiom_result = axiom_to_puml(iofonto[input])
print(axiom_result)



[Core.InformationContentEntity & obo.BFO_0000110.some(Core.MeasuredValueExpression) & Core.describes.some(obo.BFO_0000008 | obo.BFO_0000020 | Core.ProcessCharacteristic) & Core.isAbout.some(obo.BFO_0000015 | obo.BFO_0000035 | (obo.BFO_0000004 & Not(obo.BFO_0000006))) & Core.isOutputOf.some(Core.MeasurementProcess)]
1 processing Core.MeasurementInformationContentEntity
['!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml', 'class(c1, iof:MeasurementInformationContentEntity)']
2 processing Core.InformationContentEntity
['!include https://raw.githubusercontent.com/iofoundry/ontopuml/main/iof.iuml', 'class(c1, iof:MeasurementInformationContentEntity)', 'class(c2, iof:InformationContentEntity)']
3
Restriction obo.BFO_0000110.some(Core.MeasuredValueExpression)
Res Propname bfo:hasContinuantPartAtAllTimes
restriction else obo.BFO_0000110.some(Core.MeasuredValueExpression)
3 processing Core.MeasuredValueExpression
['!include https://raw.githubusercontent.com/iofoundry/

In [ ]:
print(iofonto["ProductProductionProcess"])
axiom = iofonto["ProductProductionProcess"].equivalent_to[0]
print(axiom)
print(axiom.Classes)
print(type(axiom.Classes[0]), axiom.Classes[0])
print(type(axiom.Classes[1]), axiom.Classes[1])
# print(type(axiom.Classes[2]), axiom.Classes[2])
print(type(axiom.Classes[1].value), axiom.Classes[1].value)
print(type(axiom.Classes[1].property), axiom.Classes[1].property)
# print(type(axiom.Classes[2].value), axiom.Classes[2].value)
# print(type(axiom.Classes[2].value.Classes), axiom.Classes[2].value.Classes)
# print(type(axiom.Classes[2].value.Classes[0]), axiom.Classes[2].value.Classes[0])


In [ ]:
axiom = iofonto["MeasurementInformationContentEntity"].equivalent_to[0]
print(axiom)
l = 1

print(axiom.Classes[l])
print(axiom.Classes[l].Classes)
print(axiom.Classes[l].Classes[0])
print(axiom.Classes[l].Classes[0].value.Classes[1])
print(axiom.Classes[l].Classes[0].value.Classes[1].Classes[0])
print(axiom.Classes[l].Classes[0].value.Classes[1].Classes[0].value)
print(axiom.Classes[l].Classes[0].value.Classes[1].Classes[0].value.Classes[1])

check =axiom.Classes[l].Classes[0].value.Classes[1].Classes[0].value.Classes[1].property
check.property.name

print(type(axiom))


In [ ]:
iofonto = get_ontology("https://spec.industrialontologies.org/ontology/core/Core/").load()
bfoonto = get_ontology("http://purl.obolibrary.org/obo/")
a = bfoonto.BFO_0000015


print(*a.disjoints()) #disjointwith
# print(*a.subclasses())
print(*a.is_a) #subclass of

gcas = list(iofonto.general_class_axioms())
for gca in gcas:
    if gca.is_a[0] == iofonto.ActionSpecification: 
        print("GCA",gca.left_side)
    else:
       pass 




In [ ]:
print(axiom.Classes[l].value.Classes[2])
isinstance(axiom.Classes[l].value.Classes[2],owlready2.class_construct.And)

In [ ]:
class_list = []

while isinstance(axiom.Classes[2].value.Classes[0], Restriction):
    print('Restriction')
    print(axiom.Classes[2].value.Classes[0].property)
    print(axiom.Classes[2].value.Classes[0].value)
    class_list.append(axiom.Classes[2].value.Classes[0].value)
    axiom = axiom.Classes[2].value.Classes[0].value.equivalent_to[0]
    print(axiom.Classes[0])
    print(axiom.Classes[1])
    print(axiom.Classes[2])

In [ ]:
a = iofonto["GainofRole"]
print("none" if a is None else a)
print(dir(a))
print(a.equivalent_to)